### <u> Generate the sunrise to sunset sonification used in the "_Audible Universe_" planetarium show </u>

**First, import relevant modules:**

In [ ]:
%matplotlib inline
%reload_ext autoreload 
%autoreload 2
import matplotlib.pyplot as plt
import ffmpeg as ff
import wavio as wav
from strauss.sonification import Sonification
from strauss.sources import Objects
from strauss import channels
from strauss.score import Score
import numpy as np
from strauss.generator import Sampler
import IPython.display as ipd
import glob
import os
import copy

First we download the samples to the local data directory, if they haven't been already:

In [ ]:
outdir = "../data/samples/day_sequence/"

if glob.glob(f"{outdir}/*.wav"):
    print(f"Directory {outdir} already exists.")
else:
    print("Downloading files...")
    import urllib.request
    import os
    
    path = os.path.realpath('../data/samples/day_sequence/')
    os.mkdir(path)
    
    files = ("sun_A4.wav", "scatter_B4.wav")
    urls = ("https://drive.google.com/uc?export=download&id=15D7xHEKtKppTvzzwECIq_0UGhifdhrEy",
            "https://drive.google.com/uc?export=download&id=1bnhZ_kagtWMUkj1VtEE6vzQGfnYexQfL")
    for f, u in zip(files, urls):
        with urllib.request.urlopen(u) as response, open(f"{path}/{f}", 'wb') as out_file:
            print(f"\t getting {f}")
            data = response.read() # a `bytes` object
            out_file.write(data)
    print("Done.")
    

**Specify the audio system to use** _(use `'stereo'` by default but for the planetarium `'5.1'` is used)_

In [ ]:
# specify audio system (e.g. mono, stereo, 5.1, ...)
system = "stereo"

**Now, set-up the sampler:**

In [ ]:
# set up sampler
sampler = Sampler("../data/samples/day_sequence")
sampler.modify_preset({'filter':'on'}) # want filtering on for sun altitude effect

**Set mapping limits of mapped quantities**

In [ ]:
maplims =  {'phi': (0, 360),
            'theta': (0, 180),
            'pitch' : (0, 1),
            'cutoff' : (0, 1),
            'volume' : (0,1),
            'time_evo' : (0,147)}

**Initialise the score:**

In [ ]:
# setup score
score =  Score([['A4','B4']], 147)

**Render sonification for specified planet...**

In [ ]:
data = {'phi': np.array([90,90, 0, 330, 240,240]),
        'theta': np.array([45,45,0, 40, 0, 0]), # constant theta of 90 deg
        'pitch': 1,     # constant pitch
        'volume': np.ones(6),
        'cutoff': np.array([0.5, 0.5, 1, 0.444, 0, 0]),
        'time_evo': np.array([0, 33.5,45, 57.5, 72.5, 147])}

# set up source
events = Objects(maplims.keys())
events.fromdict(data)
events.apply_mapping_functions(map_lims=maplims)

soni = Sonification(score, events, sampler, system)
soni.render()

**Listen to and plot the waveforms from the sonification:**

In [ ]:
soni.notebook_display()

In [ ]:
data2 = {'phi': np.ones(8)*0,
        'theta': np.zeros(8), # constant theta of 90 deg
        'pitch': 1,     # constant pitch
        'volume': np.array([0.2,0.2,0.4,0.2,0.1,0.03, 0.01, 0.]),
        'cutoff': np.ones(8),
        'time_evo': np.array([0, 33.5,45, 57.5, 72.5, 90, 100, 147])}

# set up source
events2 = Objects(maplims.keys())
events2.fromdict(data2)
events2.apply_mapping_functions(map_lims=maplims)

sampler2 = copy.deepcopy(sampler)
sampler2.samples['A4'] = sampler2.samples['B4']

soni2 = Sonification(score, events2, sampler2, system)
soni2.out_channels = soni.out_channels
soni2.render()

In [ ]:
soni2.notebook_display()

**Combine and save sonification to a multi-channel wav** 

NOTE: Change `"../../FILENAME.wav"` to your filepath of choice. By default, the sound file is normalised to that of the highest amplitude sample, but can be set to a lower normalisation by setting the `master_volume` parameter to a value between `0.` and `1.`.

In [ ]:
soni2.save_combined("../../day_sequence.wav", True, master_volume=1.0)